### Use this notebook for developing mlframe prototype 2

In [1]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re
import sys

In [23]:
# Class for executing commands in background processes.
# command_and_args can be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BashExecutor(multiprocessing.Process):
    
    def __init__(self, command_and_args, debug=False):
        super(BashExecutor,self).__init__()
        self.debug = debug
        if self.debug: print "command args:",command_and_args
        if type(command_and_args) is str:
            self.command = command_and_args.split(" ")
        else:
            self.command=command_and_args
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        manager = multiprocessing.Manager()
        d = manager.dict()
        d["stdout"] = ""
        d["exitcode"] = ""
        self.d = d
        
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
        if exitcode is not None:
            self.setExitCode(exitcode)
        return exitcode
    
    
    # Set given exit code to Command class object
    def setExitCode(self, ec):
        if self.d["exitcode"] == "":
            if self.debug: print "setting exit code to",ec        
            self.d["exitcode"] = ec
            if False: #isinstance(self.command, Command): 
                if self.command.exitcode is None:
                    self.command.exitcode = ec
                if self.debug:
                    print "Set exit code to",ec
                    if ec is not None:
                        printObj(self.command)

    # New version without multiprocessing 
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, shell=False)
        self.proc = proc
        if self.debug: print self.name, "process started"
        std = ""
        for std in iter(proc.stdout.readline, b''):
            self.d["stdout_empty"]=False
            if std is not None and len(std) > 0:
                self.d["stdout"] += std
                print std,
            else:
                self.d["stdout_empty"]=True
            
            time.sleep(.5)
        print "Exit code:",self.poll()
            

    def getExitcode(self):
        return self.d["exitcode"]

In [24]:
d1 = BashExecutor("/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh",debug=True)
d1.start()
#d2 = RemoteExec("mouse","test_nvidia_ubuntu.sh", debug=True)
#RemoteExec("DL","test_nvidia_ubuntu.sh", debug=False)

command args: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
In BashExecutor-17. Calling ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
BashExecutor-17 process started
Running test command on peters-imac.local
peters-imac.local():1
output 1
peters-imac.local err1
peters-imac.local():2
output 2
peters-imac.local err2
peters-imac.local():3
output 3
peters-imac.local err3
peters-imac.local():4
output 4
peters-imac.local err4
peters-imac.local():5
output 5
peters-imac.local err5
Exit code: setting exit code to 125
125


In [25]:
print d1.getExitcode()

125


In [26]:
print d1.d["stdout"]

Running test command on peters-imac.local
peters-imac.local():1
output 1
peters-imac.local err1
peters-imac.local():2
output 2
peters-imac.local err2
peters-imac.local():3
output 3
peters-imac.local err3
peters-imac.local():4
output 4
peters-imac.local err4
peters-imac.local():5
output 5
peters-imac.local err5

